# Problem 1: Analysis of Particle Swarm Optimisation

In [1]:
import numpy as np
import itertools
from matplotlib import pyplot as plt
from matplotlib import cm
import pandas as pd
import seaborn as sns

The following will be the goal ("fitness") function. Here it is to be minimised.

In [2]:
def rosenbrock(pos,dim):    #this serves as a goal function
                            # Defined by f(x,y) = (a-x)^2 + b(y-x^2)^2
                            # Using here: a = 1, b= 100, optimum 0 at (1,1)
        if dim==2:
            return ((1-pos[0])**2 + 100*(pos[1] - pos[0]**2)**2)
        elif dim==1:
            return (1-pos[0])**2 
        else: 
            ros=0;
            for i in range(dim-1):
                ros=ros+100*(pos[i+1] - pos[i]**2)**2 * (1-pos[i])**2 
            return ros

The following will be a fitness function defined by the Sphere function. It will also be minimised.

In [3]:
def sphere(x, d):   # This is the sphere function
                    # Defined by f(x, d) = SUM_i(x_i)^2
                    # x here is a vector of dimension (1,d)
    fitness = 0
    for i in range(d):
        fitness = fitness + np.square(x[i])
    return fitness

The following will be a fitness function defined by the Rastrigrin function

In [4]:
def rastrigrin(x, d):   # This is the rastrigrin function
                        # Defined by f(x, d) = 10d + SUM_i((x_i)^2 -10cos(2*pi*x_i))
                        # x here is once again a vector of dimension (1, d) and d is presumably the dimensions given to the function
    fitness = 0
    sumPart = 0
    for i in range(d):
        sumPart = sumPart + (np.square(x[i])-10*np.cos(2*np.pi*x[i]))
    fitness = 10*d + sumPart
    return fitness

The following is a class defining the Particles used in the PSO

In [5]:
class Particle: # all the material that is relavant at the level of the individual particles
    
    def __init__(self, dim, minx, maxx, ffunction):
        self.ffunction = ffunction
        self.position = np.random.uniform(low=minx, high=maxx, size=dim)
        self.velocity = np.random.uniform(low=-0.1, high=0.1, size=dim)
        self.best_particle_pos = self.position
        self.dim = dim

        if(ffunction == 'sphere'):
            self.fitness = sphere(self.position,dim)
        elif(ffunction == 'rosenbrock'):
            self.fitness = rosenbrock(self.position, dim)
        elif(ffunction == 'rastrigrin'):
            self.fitness = rastrigrin(self.position, dim)

        self.best_particle_fitness = self.fitness   # we couldd start with very large number here, 
                                                    #but the actual value is better in case we are lucky 
                
    def setPos(self, pos):
        self.position = pos

        if(self.ffunction == 'sphere'):
            self.fitness = sphere(self.position,self.dim)
        elif(self.ffunction == 'rosenbrock'):
            self.fitness = rosenbrock(self.position, self.dim)
        elif(self.ffunction == 'rastrigrin'):
            self.fitness = rastrigrin(self.position, self.dim)
        
        if self.fitness<self.best_particle_fitness:     # to update the personal best both 
                                                        # position (for velocity update) and
                                                        # fitness (the new standard) are needed
                                                        # global best is update on swarm leven
            self.best_particle_fitness = self.fitness
            self.best_particle_pos = pos

    def updateVel(self, inertia, a1, a2, best_self_pos, best_swarm_pos):
                # Here we use the canonical version
                # V <- inertia*V + a1r1 (peronal_best - current_pos) + a2r2 (global_best - current_pos)
        cur_vel = self.velocity
        r1 = np.random.uniform(low=0, high=1, size = self.dim)
        r2 = np.random.uniform(low=0, high=1, size = self.dim)
        a1r1 = np.multiply(a1, r1)
        a2r2 = np.multiply(a2, r2)
        best_self_dif = np.subtract(best_self_pos, self.position)
        best_swarm_dif = np.subtract(best_swarm_pos, self.position)
                    # the next line is the main equation, namely the velocity update, 
                    # the velocities are added to the positions at swarm level
        new_vel = inertia*cur_vel + np.multiply(a1r1, best_self_dif) + np.multiply(a2r2, best_swarm_dif)
        self.velocity = new_vel
        return new_vel

Below is the PSO class where the algorithm will run

In [6]:
class PSO: # all the material that is relavant at swarm level

    def __init__(self, w, a1, a2, dim, population_size, time_steps, search_range, ffunction):

        # Here we use values that are (somewhat) known to be good
        # There are no "best" parameters (No Free Lunch), so try using different ones
        # There are several papers online which discuss various different tunings of a1 and a2
        # for different types of problems
        self.w = w # Inertia
        self.a1 = a1 # Attraction to personal best
        self.a2 = a2 # Attraction to global best
        self.dim = dim
        self.ffunction = ffunction

        self.swarm = [Particle(dim,-search_range,search_range, ffunction) for i in range(population_size)]
        self.time_steps = time_steps
        # print('init')

        # Initialising global best, you can wait until the end of the first time step
        # but creating a random initial best and fitness which is very high will mean you
        # do not have to write an if statement for the one off case
        self.best_swarm_pos = np.random.uniform(low=-500, high=500, size=dim)
        self.best_swarm_fitness = 1e100

    def run(self):
        for t in range(self.time_steps):
            for p in range(len(self.swarm)):
                particle = self.swarm[p]

                new_position = particle.position + particle.updateVel(self.w, self.a1, self.a2, particle.best_particle_pos, self.best_swarm_pos)
                                
                if new_position@new_position > 1.0e+18: # The search will be terminated if the distance 
                                                        # of any particle from center is too large
                    # print('Time:', t,'Best Pos:',self.best_swarm_pos,'Best Fit:',self.best_swarm_fitness)
                    self.best_swarm_fitness = 1000
                    return self.best_swarm_fitness
                    # raise SystemExit('Most likely divergent: Decrease parameter values')
 
                self.swarm[p].setPos(new_position)

                if(self.ffunction == 'sphere'):
                    new_fitness = sphere(new_position,self.dim)
                elif(self.ffunction == 'rosenbrock'):
                    new_fitness = rosenbrock(new_position, self.dim)
                elif(self.ffunction == 'rastrigrin'):
                    new_fitness = rastrigrin(new_position, self.dim)

                if new_fitness < self.best_swarm_fitness:   # to update the global best both 
                                                            # position (for velocity update) and
                                                            # fitness (the new group norm) are needed
                    self.best_swarm_fitness = new_fitness
                    self.best_swarm_pos = new_position

            # if t % 100 == 0: #we print only two components even it search space is high-dimensional
                # print("Time: %6d,  Best Fitness: %14.6f,  Best Pos: %9.4f,%9.4f" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1]), end =" ")
                # if self.dim>2: 
                    # print('...')
                # else:
                    # print('')
        return self.best_swarm_fitness

Here we will be utilisation two different fitness functions, the 'sphere' function and the 'rastrigrin' function. Our job is to find parameters w and a such that the PSO algorithm can obtain good results in a short time. To achieve this, we will create a search space using w and a as the dimensions.

In [13]:
ww = np.linspace(0, 1, 26)
aa = np.linspace(0.5, 3, 26)
best_fitnesses = np.empty([26,26])
ffunction = 'rastrigrin'
time_steps = 100
for i, w in enumerate(ww):
    for j, a in enumerate(aa):
        best_fitnesses[i,j] = PSO(dim=3, w=w, a1=a, a2=a, population_size=30, time_steps=time_steps, search_range=5.12, ffunction = ffunction).run() # Modified the PSO.run() function to return the best swarm fitness
        # print("w = {}, a = {} -> Best Fitness = {}".format(w, a, best_fitnesses[i, j]))


Now plotting the data as a heatmap to visualise the distribution and quality of solutions

In [ ]:
# Plot data as heat map
data = pd.DataFrame(best_fitnesses, index = ww, columns = np.round(aa, 3))
ax = sns.heatmap(data, mask=(best_fitnesses > 1),cmap=sns.color_palette("rocket_r", as_cmap=True), cbar_kws={'label': 'Fitness'})
ax.set_facecolor('black')
plt.xlabel("$α$", fontsize = 16)
plt.ylabel("$ω$", fontsize = 16)
plt.title("{} function fitness values over {} time steps".format(ffunction.capitalize(), time_steps), fontsize = 14)
plt.tight_layout()
ax.grid(False)
# plt.savefig(r"D:\User Folders\Ben\Desktop\University\Year 5\Semester 1\NAT\media\{}{}.pdf".format(ffunction, time_steps))
plt.show()



# Problem 2: Scaling

For this problem, the dimensionality of the search space will be used as an additional paramater. For this experiment, each function will be run with w and a values which produced good results in the previous experiment

In [ ]:

best_fitnesses = np.empty([51, 51])
population_size = np.arange(1, 51, 1)
print(population_size)
ffunction = 'rastrigrin'
if(ffunction == 'sphere'):
    w = 2.4
    a = 1
    time_steps = 5
elif(ffunction == 'rastrigrin'):
    w = 0.48
    a = 1.6
    time_steps = 300

# print("Function: {}\nw: {}\na: {}\nTime steps: {}".format(ffunction, w, a, time_steps))
for i in range(51):
    for j, p in enumerate(population_size):
        best_fitnesses[i, j] = PSO(dim=3, w=w, a1=a, a2=a, population_size=p, time_steps=time_steps, search_range=5.12, ffunction = ffunction).run() # Modified the PSO.run() function to return the best swarm fitness
data = pd.DataFrame(data=best_fitnesses).melt()
data.columns = ["Population Size", "Fitness"]


After collecting the best fitnesses, display them as a line graph

In [ ]:
if(ffunction == 'sphere'):
    colour = "tomato"
elif(ffunction == 'rastrigrin'):
    colour = "mediumpurple"
graph = sns.lineplot(data = data, linewidth = 2, x = "Population Size", y = "Fitness", color = colour)
graph.set(title="Average {} function fitness values for population size".format(ffunction.capitalize()))

fig = graph.get_figure()
plt.tight_layout()
fig.savefig(r"D:\User Folders\Ben\Desktop\University\Year 5\Semester 1\NAT\media\{}pop{}.pdf".format(ffunction.capitalize(), time_steps))





# Problem 3: Heterogenous particle swarms

In this problem, we will investigate the effect of having two different kinds of particles present in the swarm for the PSO algorithm using the Rastrigrin fitness function.

In [14]:
# Modified Particle class (Heterogeneous Swarms)

class Particle: # all the material that is relavant at the level of the individual particles
    
    def __init__(self, dim, minx, maxx, ffunction):
        self.ffunction = ffunction
        self.position = np.random.uniform(low=minx, high=maxx, size=dim)
        self.velocity = np.random.uniform(low=-0.1, high=0.1, size=dim)
        self.best_particle_pos = self.position
        self.dim = dim

        if(ffunction == 'sphere'):
            self.fitness = sphere(self.position,dim)
        elif(ffunction == 'rosenbrock'):
            self.fitness = rosenbrock(self.position, dim)
        elif(ffunction == 'rastrigrin'):
            self.fitness = rastrigrin(self.position, dim)

        self.best_particle_fitness = self.fitness   # we couldd start with very large number here, 
                                                    #but the actual value is better in case we are lucky 
                
    def setPos(self, pos):
        self.position = pos

        if(self.ffunction == 'sphere'):
            self.fitness = sphere(self.position,self.dim)
        elif(self.ffunction == 'rosenbrock'):
            self.fitness = rosenbrock(self.position, self.dim)
        elif(self.ffunction == 'rastrigrin'):
            self.fitness = rastrigrin(self.position, self.dim)
        
        if self.fitness<self.best_particle_fitness:     # to update the personal best both 
                                                        # position (for velocity update) and
                                                        # fitness (the new standard) are needed
                                                        # global best is update on swarm leven
            self.best_particle_fitness = self.fitness
            self.best_particle_pos = pos

    def updateVel(self, w1, a11, a21, w2, a12, a22, best_self_pos, best_swarm_pos, oddParticle):
                # Here we use the canonical version
                # V <- inertia*V + a1r1 (peronal_best - current_pos) + a2r2 (global_best - current_pos)
        # Half the particles will be of a particular type, and the other half will be the other type
        if(oddParticle == True):
            inertia = w1
            a1 = a11
            a2 = a21
        elif(oddParticle == False):
            inertia = w2
            a1 = a12
            a2 = a22
        cur_vel = self.velocity
        r1 = np.random.uniform(low=0, high=1, size = self.dim)
        r2 = np.random.uniform(low=0, high=1, size = self.dim)
        a1r1 = np.multiply(a1, r1)
        a2r2 = np.multiply(a2, r2)
        best_self_dif = np.subtract(best_self_pos, self.position)
        best_swarm_dif = np.subtract(best_swarm_pos, self.position)
                    # the next line is the main equation, namely the velocity update, 
                    # the velocities are added to the positions at swarm level
        new_vel = inertia*cur_vel + np.multiply(a1r1, best_self_dif) + np.multiply(a2r2, best_swarm_dif)
        self.velocity = new_vel
        return new_vel

In [11]:
# Modified PSO algorithm (Heterogeneous Swarms)

class PSO: # all the material that is relavant at swarm level

    def __init__(self, w1, a11, a21, w2, a12, a22, dim, population_size, time_steps, search_range, ffunction):

        # Here we use values that are (somewhat) known to be good
        # There are no "best" parameters (No Free Lunch), so try using different ones
        # There are several papers online which discuss various different tunings of a1 and a2
        # for different types of problems
        self.w1 = w1 # Inertia 1
        self.w2 = w2 # Inertia 2
        self.a11 = a11 # Attraction to personal best 1
        self.a21 = a21 # Attraction to global best 1
        self.a12 = a12 # Attraction to personal best 2
        self.a22 = a22 # Attraction to global best 2
        self.dim = dim
        self.ffunction = ffunction

        self.swarm = [Particle(dim,-search_range,search_range, ffunction) for i in range(population_size)]
        self.time_steps = time_steps
        # print('init')

        # Initialising global best, you can wait until the end of the first time step
        # but creating a random initial best and fitness which is very high will mean you
        # do not have to write an if statement for the one off case
        self.best_swarm_pos = np.random.uniform(low=-500, high=500, size=dim)
        self.best_swarm_fitness = 1e100

    def run(self):
        for t in range(self.time_steps):
            oddParticle = True
            for p in range(len(self.swarm)):
                particle = self.swarm[p]
                new_position = particle.position + particle.updateVel(self.w1, self.a11, self.a21, self.w2, self.a12, self.a22, particle.best_particle_pos, self.best_swarm_pos, oddParticle)
                oddParticle = not oddParticle                
                if new_position@new_position > 1.0e+18: # The search will be terminated if the distance 
                                                        # of any particle from center is too large
                    # print('Time:', t,'Best Pos:',self.best_swarm_pos,'Best Fit:',self.best_swarm_fitness)
                    self.best_swarm_fitness = 1000
                    return self.best_swarm_fitness
                    # raise SystemExit('Most likely divergent: Decrease parameter values')
 
                self.swarm[p].setPos(new_position)

                if(self.ffunction == 'sphere'):
                    new_fitness = sphere(new_position,self.dim)
                elif(self.ffunction == 'rosenbrock'):
                    new_fitness = rosenbrock(new_position, self.dim)
                elif(self.ffunction == 'rastrigrin'):
                    new_fitness = rastrigrin(new_position, self.dim)

                if new_fitness < self.best_swarm_fitness:   # to update the global best both 
                                                            # position (for velocity update) and
                                                            # fitness (the new group norm) are needed
                    self.best_swarm_fitness = new_fitness
                    self.best_swarm_pos = new_position

            # if t % 100 == 0: #we print only two components even it search space is high-dimensional
                # print("Time: %6d,  Best Fitness: %14.6f,  Best Pos: %9.4f,%9.4f" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1]), end =" ")
                # if self.dim>2: 
                    # print('...')
                # else:
                    # print('')
        return self.best_swarm_fitness

We will begin by assigning a11 and a21 to be the same value, similarly, a12 and a22 will be the same value.

In [33]:
ww2 = np.linspace(0, 1, 26)
w1 = 0.48
a1 = 1.6
aa2 = np.linspace(0.5, 3, 26)
best_fitnesses = np.empty([26,26])
ffunction = 'rastrigrin'
time_steps = 300
particles = "heterogeneous"
for i, w2 in enumerate(ww2):
    for j, a2 in enumerate(aa2):

        if(particles == "homogeneous"):
            w1 = w2
            a1 = a2

        best_fitnesses[i,j] = PSO(dim=3, w1=w1, a11=a1, a21=a1, w2=w2, a12=a2, a22=a2, population_size=30, time_steps=time_steps, search_range=5.12, ffunction = ffunction).run() # Modified the PSO.run() function to return the best swarm fitness
        # print("w = {}, a = {} -> Best Fitness = {}".format(w, a, best_fitnesses[i, j]))


In [ ]:
# Plot data as heat map
data = pd.DataFrame(best_fitnesses, index = ww2, columns = np.round(aa2, 3))
ax = sns.heatmap(data, mask=(best_fitnesses > 1),cmap=sns.color_palette("rocket_r", as_cmap=True), cbar_kws={'label': 'Fitness'})
ax.set_facecolor('black')
plt.xlabel("$α$", fontsize = 16)
plt.ylabel("$ω$", fontsize = 16)
plt.title("{} function fitness for {} swarm".format(ffunction.capitalize(), particles), fontsize = 14)
plt.tight_layout()
ax.grid(False)
plt.savefig(r"D:\User Folders\Ben\Desktop\University\Year 5\Semester 1\NAT\media\{}{}{}.pdf".format(ffunction, time_steps, particles))
plt.show()


# Problem 4: Differential Evolution

Here we will use scipy's differential evolution optimiser. We will begin by recreating the Rastrigrin function such that it can be used by the optimiser, then continuing by comparing its performance against the PSO implementation.

In [77]:
from scipy.optimize import differential_evolution
bounds = [(-5.12, 5.12), (-5.12, 5.12), (-5.12, 5.12)]
# Use default mutation and recombination rates
def rastrigrinDE(x): #Hard code in the dimensionality as scipy.de inputs only x vector
    sumPart = 0
    fitness = 0
    for i in range(3):
        sumPart = sumPart + (np.square(x[i])-10*np.cos(2*np.pi*x[i]))
    fitness = 10*3 + sumPart
    return fitness

best_fitnessesDE = np.empty([51,51])
population = np.arange(1, 51, 1)
for i in range(51):
    print("Iteration [{}/50]".format(i), end="\r")
    for j, p in enumerate(population):
        best_fitnessesDE[i, j] = differential_evolution(rastrigrinDE, bounds, popsize=p, maxiter=300, mutation = 0.8, recombination = 0.9).fun

dataDE = pd.DataFrame(data=best_fitnessesDE)
dataDE = dataDE.melt()
dataDE.columns = ["Population Size", "Fitness"]

In [ ]:
ffunction = "differential evolution"

graph = sns.lineplot(data = dataDE, linewidth = 2, x = "Population Size", y = "Fitness", color = "tomato")
graph.set(title="Average {} function fitness values for population size".format(ffunction.capitalize()))

fig = graph.get_figure()
plt.tight_layout()
fig.savefig(r"D:\User Folders\Ben\Desktop\University\Year 5\Semester 1\NAT\media\{}pop.pdf".format(ffunction.capitalize()))


# Problem 5: Genetic Programming

Import relevant libraries

In [2]:
from random import random, randint, seed
from statistics import mean
from copy import deepcopy
import numpy as np

Set the parameters

In [32]:
POP_SIZE        = 600  # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 5    # maximal initial random tree depth
GENERATIONS     = 250  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 50   # size of tournament for tournament selection
XO_RATE         = 0.8  # crossover rate 
PROB_MUTATION   = 0.2  # per-node mutation probability 

Define the functions

In [4]:
def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y
def cos(x, y): return np.cos(x * y) # Notebook given to us was not built with functions with an arity of 1 in mind.

Define set of terminals

In [5]:
FUNCTIONS = [add, sub, mul, cos]
TERMINALS = ['x', 10, 2*np.pi]

Define target function, in this case the 1-D Rastrigrin function

In [20]:
ffunction = 'rastrigrin2D'

def target_func(x): # evolution's target. USING 1 DIMENSIONAL RASTRIGRIN FUNCTION
    # As the Rastrigrin function used is one dimensional, no summation necessary
    # +(10, -( * (x,x), * (10, cos( * (2, * (pi,x) ) ) ) ) )
    if(ffunction == 'rastrigrin1D'):
        return 10 + x**2 - 10*np.cos(2*np.pi*x)
    elif(ffunction == 'rastrigrin2D'):
        x2 = randint(-100, 101)
        x2 /= 100
        return 20 + x**2 - 10*np.cos(2*np.pi*x) + x2**2 - 10*np.cos(2*np.pi*x2)

Generate data points to use in the GP

In [21]:
def generate_dataset(): # generate 101 data points from target_func
    dataset = []
    for x in range(-100,101,2): 
        x /= 100
        dataset.append([x, target_func(x)])
    return dataset

Define the GP Tree building algorithm

In [22]:
class GPTree:
    def __init__(self, data = None, left = None, right = None):
        self.data  = data
        self.left  = left
        self.right = right
        
    def node_label(self): # string label
        if (self.data in FUNCTIONS):
            return self.data.__name__
        else: 
            return str(self.data)
    
    def print_tree(self, prefix = ""): # textual printout
        print("%s%s" % (prefix, self.node_label()))        
        if self.left:  self.left.print_tree (prefix + "   ")
        if self.right: self.right.print_tree(prefix + "   ")

    def compute_tree(self, x): 
        if (self.data in FUNCTIONS):
            return self.data(self.left.compute_tree(x), self.right.compute_tree(x))
        elif self.data == 'x': return x
        else: return self.data
            
    def random_tree(self, grow, max_depth, depth = 0): # create random tree using either grow or full method
        if depth < MIN_DEPTH or (depth < max_depth and not grow): 
            self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        elif depth >= max_depth:   
            self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
        else: # intermediate depth, grow
            if random () > 0.5: 
                self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
            else:
                self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        if self.data in FUNCTIONS:
            self.left = GPTree()          
            self.left.random_tree(grow, max_depth, depth = depth + 1)    
            self.right = GPTree()
            self.right.random_tree(grow, max_depth, depth = depth + 1)

    def mutation(self):
        if random() < PROB_MUTATION: # mutate at this node
            self.random_tree(grow = True, max_depth = 2)
        elif self.left: self.left.mutation()
        elif self.right: self.right.mutation() 

    def size(self): # tree size in nodes
        if self.data in TERMINALS: return 1
        l = self.left.size()  if self.left  else 0
        r = self.right.size() if self.right else 0
        return 1 + l + r

    def build_subtree(self): # count is list in order to pass "by reference"
        t = GPTree()
        t.data = self.data
        if self.left:  t.left  = self.left.build_subtree()
        if self.right: t.right = self.right.build_subtree()
        return t
                        
    def scan_tree(self, count, second): # note: count is list, so it's passed "by reference"
        count[0] -= 1            
        if count[0] <= 1: 
            if not second: # return subtree rooted here
                return self.build_subtree()
            else: # glue subtree here
                self.data  = second.data
                self.left  = second.left
                self.right = second.right
        else:  
            ret = None              
            if self.left  and count[0] > 1: ret = self.left.scan_tree(count, second)  
            if self.right and count[0] > 1: ret = self.right.scan_tree(count, second)  
            return ret

    def crossover(self, other): # xo 2 trees at random nodes
        if random() < XO_RATE:
            second = other.scan_tree([randint(1, other.size())], None) # 2nd random subtree
            self.scan_tree([randint(1, self.size())], second) # 2nd subtree "glued" inside 1st tree

Define fitness and selection process, in this case using a tournament based selection

In [23]:
def fitness(individual, dataset): # inverse mean absolute error over dataset normalized to [0,1]
    return 1 / (1 + mean([abs(individual.compute_tree(ds[0]) - ds[1]) for ds in dataset]))

def selection(population, fitnesses): # select one individual using tournament selection
    tournament = [randint(0, len(population)-1) for i in range(TOURNAMENT_SIZE)] # select tournament contenders
    tournament_fitnesses = [fitnesses[tournament[i]] for i in range(TOURNAMENT_SIZE)]
    return deepcopy(population[tournament[tournament_fitnesses.index(max(tournament_fitnesses))]]) 

Initialise the starting population

In [ ]:
def init_population(): # ramped half-and-half
    pop = []
    for md in range(3, MAX_DEPTH + 1):
        for i in range(int(POP_SIZE/6)):
            t = GPTree()
            t.random_tree(grow = True, max_depth = md) # grow
            pop.append(t) 
        for i in range(int(POP_SIZE/6)):
            t = GPTree()
            t.random_tree(grow = False, max_depth = md) # full
            pop.append(t) 
    return pop

print(len(init_population()))

Run the Genetic Programming algorithm!

In [ ]:
dataset = generate_dataset()
population = init_population() 
best_of_run = None
best_of_run_f = 0
best_of_run_gen = 0
fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]
print("Number of generations: {}\nPopulation size: {}".format(GENERATIONS, POP_SIZE))
# go evolution!
for gen in range(GENERATIONS):        
    nextgen_population=[]
    for i in range(POP_SIZE):
        parent1 = selection(population, fitnesses)
        parent2 = selection(population, fitnesses)
        parent1.crossover(parent2)
        parent1.mutation()
        nextgen_population.append(parent1)
    population=nextgen_population
    fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]
    if max(fitnesses) > best_of_run_f:
        best_of_run_f = max(fitnesses)
        best_of_run_gen = gen
        best_of_run = deepcopy(population[fitnesses.index(max(fitnesses))])
        print("________________________")
        print("gen:", gen, ", best_of_run_f:", round(max(fitnesses),3), ", best_of_run:") 
        best_of_run.print_tree()
    if (round(best_of_run_f, 3) == 1):
        print("Perfect score of {} achieved. Ending search.".format(round(best_of_run_f, 3)))
        break   

print("Total number of generations reached!")
print("\n\n_________________________________________________\nEND OF RUN\nbest_of_run attained at gen " + str(best_of_run_gen) +\
        " and has f=" + str(round(best_of_run_f,3)))
best_of_run.print_tree()